In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, concatenate, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from tensorflow.keras.models import load_model
import itertools 
filterwarnings('ignore')
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2022-05-07 09:18:40.430228: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 09:18:40.462579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 09:18:40.462756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [ ]:
import tensorflow as tf
import numpy as np
import os
import random


def set_seed(seed):
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)


seed = 1337
set_seed(seed)

In [ ]:
train_dir = "train"
val_dir = "val"
test_dir = "test"
# labels = ['benign', 'malignant']
# label_np = np.array(labels)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


datagen_train = ImageDataGenerator(width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=preprocess_input)
datagen_val = ImageDataGenerator(preprocessing_function=preprocess_input)
datagen_test = ImageDataGenerator(preprocessing_function=preprocess_input)

batch_size = 256
input_shape = (128, 128)

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)
generator_val = datagen_val.flow_from_directory(directory=val_dir,
                                                target_size=input_shape,
                                                batch_size=batch_size,
                                                shuffle=False)
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

Found 43429 images belonging to 38 classes.
Found 5417 images belonging to 38 classes.
Found 5459 images belonging to 38 classes.


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
model = MobileNetV2(include_top=False, input_shape=(128, 128, 3), weights='imagenet')
model_name = 'MobileNetV2'

2022-05-07 09:18:41.931120: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-07 09:18:41.931824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 09:18:41.932091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 09:18:41.932301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
headModel = model.output
headModel = Flatten()(headModel)
headModel = Dense(38, activation="softmax")(headModel)
model = Model(inputs=model.input, outputs=headModel)
# model.summary()

In [ ]:
def print_layer_trainable():
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


# print_layer_trainable()

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=8e-5)
loss = 'categorical_crossentropy'
metrics = ['accuracy', 'Precision', 'Recall']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
%%time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 50

callbacks = [
    EarlyStopping(patience=3, monitor='val_accuracy')
]
history = model.fit(x=generator_train,
                    epochs=epochs,
                    validation_data=generator_val,
                    callbacks=callbacks)

Epoch 1/50


2022-05-07 09:18:49.641520: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8303
2022-05-07 09:18:51.146720: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


170/170 [==============================] - 154s 862ms/step - loss: 0.7207 - accuracy: 0.8022 - precision: 0.9097 - recall: 0.7504 - val_loss: 2.0911 - val_accuracy: 0.5082 - val_precision: 0.5966 - val_recall: 0.4661
Epoch 2/50
170/170 [==============================] - 149s 873ms/step - loss: 0.1445 - accuracy: 0.9519 - precision: 0.9609 - recall: 0.9444 - val_loss: 1.3724 - val_accuracy: 0.6286 - val_precision: 0.6963 - val_recall: 0.5883
Epoch 3/50
170/170 [==============================] - 148s 868ms/step - loss: 0.0877 - accuracy: 0.9703 - precision: 0.9750 - recall: 0.9671 - val_loss: 1.0227 - val_accuracy: 0.7157 - val_precision: 0.7668 - val_recall: 0.6762
Epoch 4/50
170/170 [==============================] - 149s 873ms/step - loss: 0.0611 - accuracy: 0.9798 - precision: 0.9818 - recall: 0.9778 - val_loss: 0.7343 - val_accuracy: 0.7857 - val_precision: 0.8220 - val_recall: 0.7585
Epoch 5/50
170/170 [==============================] - 145s 849ms/step - loss: 0.0456 - accuracy: 0.